# DATASET



## PART 1 : MODIFIED VGG 16

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model

full_model = VGG16(weights='imagenet', input_shape=(None, None, 3), include_top=False)

In [ ]:
layers = [ l for l in full_model.layers]

In [ ]:
from keras.layers import Input

x = layers[0].output
to_avoid = [3, 6, 10, 19, 20, 21]
for i in range(1, len(layers)):
  if ( i in to_avoid ):
    continue
  layers[i].trainable = False
  x = layers[i](x)

model = Model(inputs = [layers[0].input], outputs = [x])

In [ ]:
model.summary()

## PART 2 : STORING ANCHORS

In [ ]:
import os
import cv2
import numpy as np
import time
import h5py

anchors = [(12,12,2,2),(19,20,3,4),(22,22,5,5),(34,35,6,7),(42,42,8,8),(49,50,9,10)]

dataset_folder = 'DATASET'
feature_folder = 'FEATURE_MAPS'

for im in os.listdir(dataset_folder):
    if 'image_' in im:
        output_name = os.path.join(feature_folder, 'feature_' + im.split('.')[0].split('_')[1] + '.hd5py')
        if os.path.isfile(output_name):
            continue
        image = cv2.imread(os.path.join(dataset_folder, im))
        image = image.reshape(1, *image.shape)
        print(image.shape)
        start = time.time()
        result = model.predict(image)
        end = time.time()
        print(result.shape, "Time taken", end-start)
        with h5py.File(output_name), 'w') as f:
            f.create_dataset("default", data=result)